In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, Input, Model, regularizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import json
import joblib

# 1. --- Load Data ---
df = pd.read_csv('all_data_labeled_final6.csv')
feature_cols = [
    'attention', 'meditation', 'delta', 'theta',
    'lowAlpha', 'highAlpha', 'lowBeta', 'highBeta',
    'lowGamma', 'highGamma', 'blinkStrength', 'time'
]
X_raw = df[feature_cols].values
y_raw = df['blinkType'].values
session_ids = df['session_id'].values

# 2. --- Feature Engineering ---
WINDOW_SIZE = 20   # Should be enough to capture two blinks for a double
STEP_SIZE = 3

def last_blink_interval(window, blink_idx, time_idx, threshold=60):
    times = window[:, time_idx]
    blinks = window[:, blink_idx] >= threshold
    blink_times = times[blinks]
    if len(blink_times) > 1:
        return blink_times[-1] - blink_times[-2]
    return 0.0

def make_window_features(X, y, session_ids, window_size=20, step_size=3):
    windows, labels, feats = [], [], []
    for start in range(0, len(X) - window_size + 1, step_size):
        end = start + window_size
        if len(np.unique(session_ids[start:end])) == 1:
            window = X[start:end]
            # Window's label: last label in window (can be 2 now if it's a double)
            label = y[end-1]
            windows.append(window)
            labels.append(label)
            blink_idx = feature_cols.index('blinkStrength')
            time_idx = feature_cols.index('time')
            blink_vals = window[:, blink_idx]
            # Feature set
            feats_window = [
                np.mean(blink_vals), np.std(blink_vals), np.min(blink_vals), np.max(blink_vals),
                blink_vals[-1], np.ptp(blink_vals),
                np.mean(np.diff(blink_vals)), np.std(np.diff(blink_vals)),
                np.sum(blink_vals > 60),
                np.sum(blink_vals == 0),
                np.min(window[:, time_idx]), np.max(window[:, time_idx]), np.ptp(window[:, time_idx])
            ]
            for band in ['delta','theta','lowAlpha','highAlpha','lowBeta','highBeta','lowGamma','highGamma']:
                idx = feature_cols.index(band)
                feats_window += [np.mean(window[:, idx]), np.std(window[:, idx])]
            feats_window.append(last_blink_interval(window, blink_idx, time_idx))
            feats.append(feats_window)
    return (np.array(windows), np.array(labels), np.array(feats))

X_win, y_win, X_feats = make_window_features(X_raw, y_raw, session_ids, WINDOW_SIZE, STEP_SIZE)

# 3. --- Scale Features ---
scaler_seq = StandardScaler()
X_win_flat = X_win.reshape(-1, X_win.shape[-1])
X_win_scaled = scaler_seq.fit_transform(X_win_flat).reshape(X_win.shape)
joblib.dump(scaler_seq, 'scaler_seq.pkl')

scaler_feats = StandardScaler()
X_feats_scaled = scaler_feats.fit_transform(X_feats)
joblib.dump(scaler_feats, 'scaler_feats.pkl')

# 4. --- Train/Test Split ---
X_temp, X_test, Xf_temp, Xf_test, y_temp, y_test = train_test_split(
    X_win_scaled, X_feats_scaled, y_win, test_size=0.15, random_state=42, stratify=y_win)
X_train, X_val, Xf_train, Xf_val, y_train, y_val = train_test_split(
    X_temp, Xf_temp, y_temp, test_size=0.1765, random_state=42, stratify=y_temp)

y_train_cat = to_categorical(y_train)
y_val_cat = to_categorical(y_val)
y_test_cat = to_categorical(y_test)

# Save for deployment:
with open('windowed_feature_cols.json', 'w') as f:
    json.dump(feature_cols, f)
with open('preprocessing_meta.json', 'w') as f:
    json.dump({'window_size': WINDOW_SIZE, 'step_size': STEP_SIZE}, f)

# 5. --- Custom Focal Loss ---
def custom_focal_loss(alpha=None, gamma=2.):
    if alpha is None:
        alpha = [0.2, 1.2, 0.6]
    alpha = tf.constant(alpha, dtype=tf.float32)
    def loss(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.clip_by_value(y_pred, tf.keras.backend.epsilon(), 1. - tf.keras.backend.epsilon())
        ce = -y_true * tf.math.log(y_pred)
        weight = alpha * tf.pow(1 - y_pred, gamma)
        loss = weight * ce
        return tf.reduce_sum(loss, axis=-1)
    return loss

# 6. --- Model: CNN + BiLSTM Hybrid ---
seq_input = Input(shape=(WINDOW_SIZE, X_win.shape[2]), name='windowed_input')
x = layers.Conv1D(32, kernel_size=3, activation='relu', padding='same')(seq_input)
x = layers.BatchNormalization()(x)
x = layers.Conv1D(32, kernel_size=3, activation='relu', padding='same')(x)
x = layers.MaxPooling1D(2)(x)
x = layers.Dropout(0.3)(x)
x = layers.Bidirectional(layers.LSTM(64, return_sequences=False, dropout=0.3))(x)
x = layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(1e-4))(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.3)(x)

feat_input = Input(shape=(X_feats.shape[1],), name='features_input')
y = layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(1e-4))(feat_input)
y = layers.BatchNormalization()(y)
y = layers.Dense(16, activation='relu')(y)

merged = layers.concatenate([x, y])
z = layers.Dense(48, activation='relu', kernel_regularizer=regularizers.l2(1e-4))(merged)
z = layers.BatchNormalization()(z)
z = layers.Dropout(0.2)(z)
output = layers.Dense(3, activation='softmax')(z)

model = Model(inputs=[seq_input, feat_input], outputs=output)
model.compile(
    optimizer='adam',
    loss=custom_focal_loss(alpha=[0.3, 1.0, 0.7], gamma=2),
    metrics=['accuracy']
)
model.summary()

# 7. --- Train ---
callbacks = [
    EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.6, min_lr=1e-5, verbose=1),
    ModelCheckpoint('best_eeg_cnn_bilstm_focal.h5', monitor='val_loss', save_best_only=True, verbose=1)
]
history = model.fit(
    [X_train, Xf_train], y_train_cat,
    epochs=100,
    batch_size=96,
    validation_data=([X_val, Xf_val], y_val_cat),
    callbacks=callbacks,
    verbose=2
)

# 8. --- Evaluate ---
model = tf.keras.models.load_model(
    'best_eeg_cnn_bilstm_focal.h5',
    custom_objects={'loss': custom_focal_loss(alpha=[0.3, 1.0, 0.7], gamma=2)}
)
y_test_pred_probs = model.predict([X_test, Xf_test])
y_test_pred = np.argmax(y_test_pred_probs, axis=1)
y_test_true = np.argmax(y_test_cat, axis=1)

print("\nClassification Report (Hybrid Model, Test Set):")
print(classification_report(y_test_true, y_test_pred, digits=4))

cm = confusion_matrix(y_test_true, y_test_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap='Blues')
plt.title("Confusion Matrix (Hybrid Model, Test Set)")
plt.show()

# 9. --- Plot Learning Curves ---
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.title('Accuracy over Epochs'); plt.legend()
plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.title('Loss over Epochs'); plt.legend()
plt.show()

# 10. --- Show Sample Predictions ---
for i in range(10):
    idx = np.random.randint(0, len(y_test_true))
    print(f"Example {i+1}: Actual: {y_test_true[idx]}, Predicted: {y_test_pred[idx]}, Softmax: {np.round(y_test_pred_probs[idx], 2)}")
